<a href="https://colab.research.google.com/github/mritunjay-07/102003030_Sampling_Assignment/blob/main/102003030_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LOADING THE DATASET AND IMPORTING OF LIBRARIES**

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('/content/drive/MyDrive/All csv/Creditcard_data.csv')
# print(data.head())
x=data.iloc[:,0:30]
y=data.iloc[:,30]

# print(x.head())
# print(y)

# **BALANCING THE DATASET**

In [ ]:
# Converting imbalanced dataset to balanced dataset-->

# Rejecting Random Under Sampling because it is causing loss of 
# information 
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=45, replacement=True)
# x_rus, y_rus = rus.fit_resample(x, y)
from collections import Counter
from imblearn.over_sampling import SMOTE
import random

smote = SMOTE()
x_smote, y_smote = smote.fit_resample(x, y)

# print('original dataset shape:', Counter(y))
# print('Resample dataset shape', Counter(y_smote))


In [ ]:
x_smote['Class']=y_smote
# print(x_smote.head())

# **CREATING FIVE SAMPLES USING DIFFERENT SAMPLING TECHNIQUES**

In [ ]:
# Sample 1 using Stratified Sampling-->
Z=1.96
p=0.5
E=0.10
S=2
n2=round((Z*Z*p*(1-p))/(E/S)**2)
# print(n2)
n2=int(n2/2)
sample1 = x_smote.groupby('Class', group_keys=False).apply(lambda x: x.sample(n2))

In [ ]:
# Sample 2 using Systematic Sampling-->
# n = len(x_smote)
# k = n//n1 
# start = np.random.randint(k)
# indices = np.arange(start, n, k)
# x_smote[indices[:n1]]
s=int(1526/96)
indexes = np.arange(0, len(x_smote), step=s)
sample2 = x_smote.iloc[indexes]
# print(sample2)

In [ ]:
# Sample 3 using Convenience Sampling-->
sorted_x_smote = x_smote.sort_values(['Amount', 'Time'],
              ascending = [True, True])
sample_size = 100
sample = sorted_x_smote.iloc[:,25:sample_size]
sample3=sample.iloc[:,1:]
# print(sample3)

In [ ]:
# Sample 4 using Cluster Sampling-->
Z=1.96
p=0.5
E=0.10
C=2
n4=round((Z*Z*p*(1-p))/(E/C)**2)
# print(n4)
def Sample_Creation(data_to_work_on, data_in_each_cluster, cluster_selection):
    N = len(data_to_work_on)
    K = int(N/data_in_each_cluster)
    data = None
    for k in range(K):
        sample_k = data_to_work_on.sample(data_in_each_cluster)
        sample_k["cluster"] = np.repeat(k,len(sample_k))
        data_to_work_on = data_to_work_on.drop(index = sample_k.index)
        data = pd.concat([data,sample_k],axis = 0)

    random_chosen_clusters = np.random.randint(0,K,size = cluster_selection)
    samples = data[data.cluster.isin(random_chosen_clusters)]
    return(samples)

sample4 = Sample_Creation(data_to_work_on = x_smote, data_in_each_cluster = 192, cluster_selection = C) 
sample4 = sample4.iloc[:,1:31]
# print(sample4)

In [ ]:
# Sample 5 using Simple Random Sampling-->
Z=1.96
p=0.5
E=0.10
n1=round((Z*Z*p*(1-p))/(E*E))
# print(n1)
sample5 = x_smote.sample(n=n1, random_state=0)
# random_sample.head()

# **APPLYING FIVE DIFFERENT MODELS ON EACH SAMPLE**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

list_of_dataframes =[sample1,sample2,sample3,sample4,sample5]
models = {'M1': LogisticRegression(), 'M2': LinearSVC(), 'M3': DecisionTreeClassifier(),
          'M4':GaussianNB(),'M5':KNeighborsClassifier()}

scaler = StandardScaler()
df_model = pd.DataFrame(index=models.keys())
accuracy = {}

for i, df in enumerate(list_of_dataframes, start=1):
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    for keys in models.keys():
        models[keys].fit(X_train, y_train)
        y_pred = models[keys].predict(X_test)

        accuracy[keys] = round(sklearn.metrics.accuracy_score(y_test, y_pred),2)
    df_model['Sample'+str(i)] = accuracy.values()

# **ACCURACY TABLE**

In [ ]:
print(df_model)

    Sample1  Sample2  Sample3  Sample4  Sample5
M1     0.95     0.86     0.66     0.90     0.90
M2     0.92     0.86     0.66     0.91     0.95
M3     0.92     0.90     0.94     0.97     0.80
M4     0.87     0.86     0.59     0.74     0.80
M5     0.87     0.67     0.88     0.86     0.80
